# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성

torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공

- PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)

- 신경망은 다른 모듈(계층; layer)로 구성된 모듈

- 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 장치 설정

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. torch.cuda 를 사용할 수 있는지 확인하고 그렇지 않으면 CPU를 계속 사용

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


# 클래스 정의

신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화

nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

In [3]:
class NeuralNetwork(nn.Module): # nn.Module의 sub class
    def __init__(self): # 신경망 계층 초기화
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # relu 활성화 함수를 활용해 간단하게 쌓음
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device 로 이동한 뒤, 구조(structure)를 출력

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 입력 데이터를 전달, 이는 일부 백그라운드 연산들(https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866) 과 함께 모델의 forward 를 실행 

model.forward() 를 직접 호출 XXX

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 10-차원 텐서가 반환

 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻는다.

In [13]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

tensor([[0.0280, 0.0941, 0.0956, 0.0000, 0.0426, 0.0370, 0.0000, 0.0000, 0.0091,
         0.0267]], grad_fn=<ReluBackward0>)
Predicted class: tensor([2])


# 모델 계층(layer)

FashionMNIST 모델의 계층들을 살펴보기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져오자.

In [14]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환 (dim=0의 미니배치 차원은 유지)

In [15]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


선형 계층 은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈

In [16]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity) 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0754, -0.0920, -0.4195, -0.0816,  0.1585,  0.1059,  0.0859, -0.0237,
         -0.4213, -0.3847,  0.2132, -0.6012, -0.0778,  0.2698,  0.1957,  0.3360,
          0.4244,  0.2096, -0.1568,  0.3513],
        [ 0.4669, -0.2308, -0.2152, -0.0756,  0.2111,  0.1579, -0.2932, -0.0664,
         -0.5924, -0.4994, -0.0131, -0.1768,  0.0149,  0.0926,  0.2496,  0.2889,
          0.6129,  0.1932, -0.1410,  0.2658],
        [-0.0569, -0.1538, -0.2790, -0.1473,  0.2239, -0.1712, -0.1137, -0.2212,
         -0.6220, -0.0491,  0.3332, -0.3355, -0.1989,  0.0284,  0.1783,  0.5151,
          0.3050, -0.1781,  0.2577,  0.4806]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0754, 0.0000, 0.0000, 0.0000, 0.1585, 0.1059, 0.0859, 0.0000, 0.0000,
         0.0000, 0.2132, 0.0000, 0.0000, 0.2698, 0.1957, 0.3360, 0.4244, 0.2096,
         0.0000, 0.3513],
        [0.4669, 0.0000, 0.0000, 0.0000, 0.2111, 0.1579, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0149, 0.0926, 0.249

nn.Sequential 은 순서를 갖는 모듈의 컨테이너

- 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달
- 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 만들 수 있다.

In [21]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [22]:
print(logits)

tensor([[ 0.2496, -0.2913,  0.3128, -0.2472, -0.0322, -0.1965,  0.1731,  0.1055,
         -0.0341, -0.3573],
        [ 0.2149, -0.2263,  0.2322, -0.2594, -0.0179, -0.2542,  0.0123, -0.0184,
          0.0583, -0.1947],
        [ 0.1998, -0.2281,  0.2364, -0.1653,  0.0105, -0.0632,  0.2019,  0.1404,
         -0.0704, -0.2384]], grad_fn=<AddmmBackward>)


In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [24]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0098, -0.0305, -0.0253,  ...,  0.0262,  0.0083,  0.0201],
        [ 0.0349, -0.0177, -0.0262,  ..., -0.0335, -0.0310, -0.0109]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0164, 0.0315], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0113,  0.0330, -0.0127,  ...,  0.0246, -0.0378, -0.0024],
        [ 0.0270,  0.0365,  0.0128,  ...,  0.0204, -0.0042,  0.0092]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_st